In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import random
import numpy as np
import pandas as pd

random.seed(1234)
torch.manual_seed(1234)
np.random.seed(1234)

from torch.utils.data import Dataset, DataLoader

import json
import glob
import itertools
from os.path import basename
from collections import *
from tqdm.notebook import tqdm
import networkx as nx
import random
import pickle
import tensorflow as tf
from collections import Counter
from tensorboardX import SummaryWriter

from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

import sklearn

from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedKFold

from ast import literal_eval

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
df = pd.read_csv('/shared/0/projects/condolence/working-dir/condolence_annotator/output.csv')

In [3]:
df.head()

,Unnamed: 0,rating,target,observer,rounded,user1,user2
0,0,1.5,Thank you. I needed to hear this. The other pr...,"This sounds like the bigger issue, to an outsi...",2,1.0,2.0
1,1,2.0,"well, I don't sleep at all anyway, so that's n...",I know a few words can't undo years of mental ...,2,3.0,1.0
2,2,1.0,"Thank you. Yes, agreed on the timing, the las...","I'm so sorry, that is all really tough. We're ...",1,1.0,1.0
3,3,1.0,There have definitely been signs that things w...,Yea. I couldn't even imagine. I have unstable ...,1,1.0,1.0
4,4,1.0,I don’t know whether anybody here will remembe...,"I am sorry this is happening, that so sad and ...",1,1.0,1.0


In [4]:
if True: 
    
    rounded_to_rows = defaultdict(list)
    for i, row in df.iterrows():
        rounded_to_rows[row['rounded']].append([row['target'], row['observer'], row['rating']])    
    
    train_df = defaultdict(list)
    eval_df = defaultdict(list)
    test_df = defaultdict(list)
    for val, rows in rounded_to_rows.items():
        random.shuffle(rows)
        for i, row in enumerate(rows):
            if i % 10 == 0:
                cur_df = test_df
            elif i % 10 == 1:
                cur_df = eval_df
            else:
                cur_df = train_df
            cur_df['text_a'].append(row[1]) # observer
            cur_df['text_b'].append("") # 
            cur_df['labels'].append(row[2]) # rating
            cur_df['target'].append("") # target
            cur_df['observer'].append(row[1]) # observer
            cur_df['rating'].append(row[2]) # rating
            
    train_df = pd.DataFrame(train_df)
    eval_df = pd.DataFrame(eval_df)
    test_df = pd.DataFrame(test_df)
            
    print(len(df), len(train_df), len(eval_df), len(test_df))
        
else:
    df['text_a'] = df['target']
    df['text_b'] = df['observer']
    df['labels'] = df['rating']
    train_df, eval_df, test_df = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))])

1004 798 103 103


In [5]:
print(len(df), len(train_df), len(eval_df), len(test_df))

1004 798 103 103


In [6]:
import warnings

def fxn():
    warnings.warn("deprecated", UserWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    
manual_seed = 1234

train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 20,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 100,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": True,
    'regression': True,
    "manual_seed": manual_seed,
    'cache_dir': '/shared/0/projects/condolence/working-dir/empathy-classifier-no-ftlm-only-reply-%d' % manual_seed,
    'output_dir': '/shared/0/projects/condolence/working-dir/empathy-classifier-no-ftlm-only-reply-%d' % manual_seed
    
}


# Create a ClassificationModel
model = ClassificationModel('roberta', 'roberta-base', num_labels=1, 
                            use_cuda=True, cuda_device=1, args=train_args)
#print(train_df.head())

# Train the model
model.train_model(train_df, eval_df=eval_df, show_running_loss=True, 
                  mse=sklearn.metrics.mean_squared_error,
                  r2=sklearn.metrics.r2_score)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

predictions, raw_outputs = model.predict([["I'd like to puts some CD-ROMS on my iPad, is that possible?'", "Yes, but wouldn't that block the screen?"]])
print(predictions)
print(raw_outputs)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE

/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)



1.0376939
1.0376939


In [7]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, mse=sklearn.metrics.mean_squared_error,
                  r2=sklearn.metrics.r2_score)
print(result)


{'mse': 0.55470574, 'r2': 0.09351925423146024, 'eval_loss': 0.5773679195688322}


# Train the regressor using a fine-tuned language model as the base parameters

In [8]:
import warnings

def fxn():
    warnings.warn("deprecated", UserWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    
manual_seed=1234
    
train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 20,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 100,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": True,
    "save_model_every_epoch": True,
    'regression': True,
    "manual_seed": manual_seed,
    'cache_dir': '/shared/0/projects/condolence/working-dir/empathy-classifier-ftlm-only-reply-%d' % manual_seed,
    'output_dir': '/shared/0/projects/condolence/working-dir/empathy-classifier-ftlm-only-reply-%d' % manual_seed    
}


model_file = '/shared/0/projects/condolence/working-dir/empathy-lm-model/checkpoint-69639-epoch-1/'

# Create a ClassificationModel
model = ClassificationModel('roberta', model_file, num_labels=1, 
                            use_cuda=True, cuda_device=1, args=train_args)
#print(train_df.head())

# Train the model
model.train_model(train_df, eval_df=eval_df, show_running_loss=True, 
                  mse=sklearn.metrics.mean_squared_error,
                  r2=sklearn.metrics.r2_score)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Some weights of the model checkpoint at /shared/0/projects/condolence/working-dir/empathy-lm-model/checkpoint-69639-epoch-1/ were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /shared/0/projects/condolence/working-dir/

/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
/opt/anaconda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [9]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df, mse=sklearn.metrics.mean_squared_error,
                  r2=sklearn.metrics.r2_score)
print(result)


{'mse': 0.56198347, 'r2': 0.08162616121290989, 'eval_loss': 0.5846960765237992}


In [10]:
result, model_outputs, wrong_predictions = model.eval_model(test_df, mse=sklearn.metrics.mean_squared_error,
                  r2=sklearn.metrics.r2_score)
print(result)


{'mse': 0.56198347, 'r2': 0.08162616121290989, 'eval_loss': 0.5846960765237992}
